In [1]:
import os
import cv2
import numpy as np
import keras
import pickle

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from imutils import paths

Library versions should be used:
keras = 2.3.1
numpy = 1.17.4


In [2]:
datapath = "D:/Sun'iyX/Data/Pictured"
outputmodel = r"D:\Sun'iyX\ML\VideoClsfImgbased\ContentTypeClassifier\video_classification_model\videoclassificationModel"
outputlabelbinarizer = r"D:\Sun'iyX\ML\VideoClsfImgbased\ContentTypeClassifier\model\videoclassificationbinarizer"

In [3]:
media_labels = set(['Ad','Content'])
print("images are being loaded ...")
 
pathToImages = list(paths.list_images(datapath))

data = []

labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in media_labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (244,244))
    data.append(image)
    labels.append(label)
        
         

images are being loaded ...


In [4]:
data = np.array(data)
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size = 0.25, stratify=labels, random_state=42 )

In [6]:
trainingAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
    
)

validationAugmentation = ImageDataGenerator()
mean = np.array([123.68, 166.779, 103.939], dtype="float32")
trainingAugmentation.mean = mean
validationAugmentation.mean = mean

In [7]:
from keras.applications import ResNet50

from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten, Dense, Dropout

from keras.models import Model


In [8]:
baseModel = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,244,3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name = "Flatten")(headModel)
headModel = Dense(512, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation='sigmoid')(headModel) # activation function changed from softmax to sigmoid 
model = Model(inputs = baseModel.input, outputs = headModel)

for baseModelLayers in baseModel.layers:   
       
    baseModelLayers.trainable = False

In [9]:
epoch = 25
# opt = SGD(lr = 0.0001, momentum=0.9, decay = 0.0001/epoch)

In [10]:
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy']) # optimizer changed to adam instead of opt  

In [11]:
History = model.fit_generator(
    trainingAugmentation.flow(X_train, Y_train, batch_size=32),
    steps_per_epoch=len(X_train)//32,
    validation_data= validationAugmentation.flow(X_test, Y_test),
    validation_freq=len(X_test)//32,
    epochs=epoch
)

C:\Users\USER1\AppData\Local\Temp\ipykernel_2940\2583291714.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  History = model.fit_generator(


Epoch 1/25
49/49 [==============================] - 68s 1s/step - loss: 0.0105 - accuracy: 0.9949
Epoch 2/25
49/49 [==============================] - 65s 1s/step - loss: 3.1756e-16 - accuracy: 1.0000
Epoch 3/25
49/49 [==============================] - 65s 1s/step - loss: 6.4528e-14 - accuracy: 1.0000
Epoch 4/25
49/49 [==============================] - 66s 1s/step - loss: 1.4991e-17 - accuracy: 1.0000
Epoch 5/25
49/49 [==============================] - 69s 1s/step - loss: 8.9393e-18 - accuracy: 1.0000
Epoch 6/25
49/49 [==============================] - 68s 1s/step - loss: 7.3574e-18 - accuracy: 1.0000
Epoch 7/25
49/49 [==============================] - 66s 1s/step - loss: 2.3710e-16 - accuracy: 1.0000
Epoch 8/25
49/49 [==============================] - 68s 1s/step - loss: 1.0059e-16 - accuracy: 1.0000
Epoch 9/25
49/49 [==============================] - 65s 1s/step - loss: 1.9237e-15 - accuracy: 1.0000
Epoch 10/25
49/49 [==============================] - 65s 1s/step - loss: 6.4432e-17 - 

In [14]:
model.save(outputmodel)
lbinarizer = open(r"D:\Sun'iyX\ML\VideoClsfImgbased\ContentTypeClassifier\model\classificationbinarizer.pickle", "wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()

INFO:tensorflow:Assets written to: D:\Sun'iyX\ML\VideoClsfImgbased\ContentTypeClassifier\video_classification_model\videoclassificationModel\assets


INFO:tensorflow:Assets written to: D:\Sun'iyX\ML\VideoClsfImgbased\ContentTypeClassifier\video_classification_model\videoclassificationModel\assets
